In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "models/gemini-2.5-flash-lite")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model=GEMINI_MODEL, api_key=GEMINI_API_KEY)

In [4]:
import chromadb

client = chromadb.PersistentClient(path="./store_emb")

collection = client.get_or_create_collection(
    name="rag_collection", 
    configuration={
        "hnsw": {
            "space": "cosine",
            "ef_construction": 200,
            "max_neighbors":32
        }
    }
)

In [8]:
import pdfplumber
import re
from collections import Counter
from pathlib import Path

def clean_pdf_text(pdf_path, min_line_length=10):
    """
    Extracts text from each page, removes headers and footers by detecting repeating lines.
    Returns list of dicts: [{ 'page': n, 'text': clean_text }]
    """
    pdf = pdfplumber.open(pdf_path)
    pages_text = []
    header_footer_candidates = []

    # Extract raw lines page-wise
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text() or ""
        lines = [line.strip() for line in text.split("\n") if len(line.strip()) > min_line_length]
        header_footer_candidates.extend(lines[:2] + lines[-2:])  # top 2 + bottom 2 lines
        pages_text.append((page_num, lines))
    
    # Detect frequently repeating lines (likely headers/footers)
    counts = Counter(header_footer_candidates)
    repetitive = {line for line, c in counts.items() if c > 0.6 * len(pdf.pages)}

    # Clean pages
    cleaned_pages = []
    for page_num, lines in pages_text:
        clean_lines = [ln for ln in lines if ln not in repetitive]
        clean_text = "\n".join(clean_lines)
        cleaned_pages.append({"page": page_num, "text": clean_text})
    pdf.close()
    return cleaned_pages


In [9]:
from langchain_core.documents import Document

def build_docs_from_pdf(pdf_path):
    cleaned_pages = clean_pdf_text(pdf_path)
    file_name = Path(pdf_path).stem
    docs = []
    for page_obj in cleaned_pages:
        page_num = page_obj["page"]
        text = page_obj["text"]
        metadata = {
            "source": file_name,
            "page_number": page_num,
            "file_path": str(pdf_path)
        }
        docs.append({"page_content": text, "metadata": metadata})
    return docs


In [10]:
import os

folder_path = "../data/"

all_pdfs = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pdf")]

In [15]:
text_documents = []

for pdf_file in [all_pdfs[-1]]:
    docs = build_docs_from_pdf(pdf_file)
    text_documents.extend(docs)

Cannot set gray non-stroke color because /'H1' is an invalid float value
Cannot set gray non-stroke color because /'H1' is an invalid float value
Cannot set gray non-stroke color because /'H1' is an invalid float value
Cannot set gray non-stroke color because /'H1' is an invalid float value
Cannot set gray non-stroke color because /'H1' is an invalid float value
Cannot set gray non-stroke color because /'H1' is an invalid float value
Cannot set gray non-stroke color because /'H1' is an invalid float value


In [16]:
text_documents

[{'page_content': 'DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via\nReinforcement Learning\nDeepSeek-AI\nresearch@deepseek.com\nWe introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.\nDeepSeek-R1-Zero,amodeltrainedvialarge-scalereinforcementlearning(RL)withoutsuper-\nvisedfine-tuning(SFT)asapreliminarystep,demonstratesremarkablereasoningcapabilities.\nThrough RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing\nreasoningbehaviors. However,itencounterschallengessuchaspoorreadability,andlanguage\nmixing. To address these issues and further enhance reasoning performance, we introduce\nDeepSeek-R1,whichincorporatesmulti-stagetrainingandcold-startdatabeforeRL.DeepSeek-\nR1achievesperformancecomparabletoOpenAI-o1-1217onreasoningtasks. Tosupportthe\nresearchcommunity,weopen-sourceDeepSeek-R1-Zero,DeepSeek-R1,andsixdensemodels\n(1.5B,7B,8B,14B,32B,70B)distilledfromDeepSeek-R1basedonQwenandLlama.\nAIME 2024 Codeforces GPQA Diam

In [59]:
# chunking.py
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter

def get_text_splitter(strategy="recursive", chunk_size=800, chunk_overlap=150, model_name="gpt-4o"):
    if strategy == "token":
        return TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, model_name=model_name)
    # default: character recursive (works well)
    return RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# usage
splitter = get_text_splitter(strategy="recursive", chunk_size=800, chunk_overlap=200)


In [ ]:
texts = splitter.split_text(text_documents[0]["page_content"])

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [84]:
import time
from tqdm import tqdm

def gemini_embedding_batch(texts, batch_size=2):
    result_emb = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        result_emb.extend(embeddings.embed_documents(batch))
        time.sleep(2)  # to avoid rate limits
    return result_emb

In [ ]:
# make chunks of dataset 


def chunking_doc(documents):
    chunking_dataset = []
    for document in documents:
        texts = splitter.split_text(document["page_content"])
        for text in texts:
            chunking_dataset.append({
                "page_content": text,
                "metadata": document["metadata"]
            })
    return chunking_dataset

In [66]:
len(text_documents)

41

In [67]:
chunking_text_document = chunking_doc(text_documents)
len(chunking_text_document)

118

In [78]:

def embed_and_store(docs):
    # chunking 
    texts = [doc['page_content'] for doc in docs]
    metadatas = [doc['metadata'] for doc in docs]
    embs = gemini_embedding_batch(texts)

    
    ids = [f"doc_{i}" for i in range(len(docs))]
    
    collection.add(
        documents=texts,
        metadatas=metadatas,
        embeddings=embs,
        ids=ids
    )

In [85]:
embed_and_store(chunking_text_document)

100%|██████████| 59/59 [02:40<00:00,  2.72s/it]


In [7]:
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma

db = Chroma(collection_name="rag_collection", client=client, embedding_function=embeddings)

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":5})

mmr_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k":5, "fetch_k":10, "lambda_mult":0.5})

from sentence_transformers import SentenceTransformer, CrossEncoder

cross_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")  # small and fast-ish

def rerank_with_cross_encoder(query, candidates):
    # candidates: list of Document-like dicts with 'page_content'
    pairs = [[query, c.page_content] for c in candidates]
    scores = cross_model.predict(pairs)
    ranked = sorted(zip(scores, candidates), key=lambda x: x[0], reverse=True)
    return [c for s, c in ranked]

llm = model

prompt = PromptTemplate.from_template("""
You are a helpful assistant. Use the retrieved documents to answer the user query.
{context}

Question: {question}
""")
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",  # or 'map_reduce' or 'refine'
                                       retriever=retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={ "prompt": prompt })



In [ ]:
clien

In [95]:
def answer_query(query):
    # Step A: get top candidates from vectordb (fetch_k large)
    docs = db._collection.query(query_embeddings=embeddings.embed_query(query), n_results=20)["documents"][0]
    # convert to Document objects if required; this access is implementation-dependent in Chroma.
    from langchain_core.documents import Document
    candidate_docs = [Document(page_content=d, metadata={}) for d in docs]
    # Step B: rerank with cross-encoder
    reranked = rerank_with_cross_encoder(query, candidate_docs)[:5]
    # Step C: merge into context
    combined_context = "\n\n---\n\n".join([d.page_content for d in reranked])
    # Step D: call LLM
    response = llm.invoke(prompt.format(context=combined_context, question=query))
    return response, reranked

In [98]:
ans, sources = answer_query("list step for transformer model deployment")

In [102]:
sources

[Document(metadata={}, page_content='Action for you: Try to implement a tiny Transformer from scratch in\nPyTorch or TensorFlow.\n2. Pretraining of LLMs\nLLMs like GPT, LLaMA, Falcon, Mistral are trained in two main stages.\nPretraining Objective\nUsually causal language modeling (CLM) → predict next token.\nOn massive datasets (hundreds of billions of tokens).\nTraining Pipeline\nTokenization (BPE, SentencePiece, WordPiece)\nDatasets (Common Crawl, books, code, Wikipedia, etc.)\nLoss Function: Cross-Entropy\nOptimization: AdamW, learning rate warm-up, cosine decay\nMixed Precision Training (FP16/BF16)\nGradient Accumulation, Gradient Clipping\nDistributed Training: Data Parallelism, Model Parallelism, Pipeline\nParallelism\nDeep dive topics: scaling laws, compute-data tradeoff, Chinchilla optimal\n3. Fine-Tuning LLMs'),
 Document(metadata={}, page_content='Stage 5: Advanced Architectures & Research\nGoal: Explore cutting-edge improvements.\nConcepts to Learn\nMixture of Experts (MoE) 

In [101]:
print(ans.content)

Based on the provided documents, there isn't a direct step-by-step guide for Transformer model deployment. However, we can infer some general stages and concepts that would be involved in deploying such a model.

Here's a breakdown of potential steps, drawing from the information about LLM training and advanced architectures:

**Inferred Steps for Transformer Model Deployment:**

1.  **Model Training/Fine-tuning (Pre-deployment Stage):**
    *   **Pretraining:** This is the initial stage where the model learns general language understanding on massive datasets. The documents mention "causal language modeling (CLM)" as a common pretraining objective.
    *   **Fine-tuning:** After pretraining, the model is often fine-tuned for specific tasks or domains. This involves using smaller, task-specific datasets and techniques like LoRA (Low-Rank Adaptation) for efficient adaptation.

2.  **Model Optimization for Inference:**
    *   **Quantization:** This is a technique to reduce the precision